# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import weather_api_key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Study data files
weather_meta_data = "my_weather_data.csv"

# Read the weather data and the study results
weather_data = pd.read_csv(weather_meta_data)

# Combine the data into a single dataset
weatherdf = pd.DataFrame(weather_data)
weatherdf

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date
0,0,Puerto Ayora,-0.7393,-90.3518,71.42,73,9.91,64,EC,1596304800
1,1,Verkhnevilyuysk,63.4453,120.3167,56.59,69,2.51,0,RU,1596304800
2,2,Maceió,-9.6658,-35.7353,77.16,79,12.10,55,BR,1596304800
3,3,Grindavik,63.8424,-22.4338,54.05,83,11.63,80,IS,1596304800
4,4,Coahuayana Viejo,18.7333,-103.6833,91.72,64,4.38,8,MX,1596304800
...,...,...,...,...,...,...,...,...,...,...
556,556,Arequipa,-16.3989,-71.5350,72.28,14,10.29,56,PE,1596304800
557,557,Nyamuswa,-1.9000,34.0167,74.48,60,4.14,23,TZ,1596304800
558,558,Westport,41.1415,-73.3579,86.76,50,1.39,8,US,1596304800
559,559,Gomel,52.4345,30.9754,61.41,64,8.43,21,BY,1596304800


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
humidity = weatherdf["Humidity"].astype(float)
maxhumidity = humidity.max()

locations = weatherdf[["Latitude", "Longitude"]]

In [5]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=humidity,
                                 dissipating=False, 
                                 max_intensity=maxhumidity,
                                 point_radius=1)
fig.add_layer(heat_layer)

plt.savefig("zaheer - vacationheatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
narrowed_city_df = weatherdf.loc[(weatherdf["Max Temp"] > 70) & (weatherdf["Max Temp"] < 80) & (weatherdf["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date
0,96,Salta,-24.7859,-65.4117,76.87,22,7.00,0,AR,1596304800
1,127,Port Hedland,-20.3167,118.5667,70.99,50,7.25,0,AU,1596304800
2,174,Lata,40.1629,-8.3327,73.17,53,8.68,0,PT,1596304800
3,190,Yabrūd,33.9674,36.6574,76.17,30,3.60,0,SY,1596304800
4,208,Mossamedes,-15.1961,12.1522,70.20,75,2.86,0,AO,1596304800


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotellist = []
for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Latitude']
    lng = narrowed_city_df.loc[i]['Longitude']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date,Hotel Name
0,96,Salta,-24.7859,-65.4117,76.87,22,7.00,0,AR,1596304800,Salta
1,127,Port Hedland,-20.3167,118.5667,70.99,50,7.25,0,AU,1596304800,Port Hedland
2,174,Lata,40.1629,-8.3327,73.17,53,8.68,0,PT,1596304800,Coimbra
3,190,Yabrūd,33.9674,36.6574,76.17,30,3.60,0,SY,1596304800,Yabrud
4,208,Mossamedes,-15.1961,12.1522,70.20,75,2.86,0,AO,1596304800,Namibe


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Latitude", "Longitude"]]

In [10]:
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
plt.savefig("zaheer - hotelmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>